In [172]:
import numpy as np
import pandas as pd
import cv2
import os
image4 = cv2.imread("data/memorial.hdr",cv2.IMREAD_ANYDEPTH)

In [173]:
#************ Q4 a************8
# Converting hsi image to rgb image
def hsi_to_rgb_value(h, s, i):
    h = h * 360
    if(h >= 240):
        Hs = h-240
    elif(h >= 120):
        Hs = h-120
    else:
        Hs = h
    a = i * (1 - s)
    b = i * (1 + s * np.cos(np.radians(Hs)) / np.cos(np.radians(60 - Hs)))
    c = 3*i - (a+b)
    if h < 120:
        b,r,g = a,b,c
    elif h < 240:
        r,g,b = a,b,c
    else:
        g,b,r = a,b,c
    return np.clip(np.array([r, g, b]), 0, 1)

# Creating an hsi image 
def create_hsi_image(image_size=256,constant='hue',val = 0.5):
    image = np.zeros((image_size,image_size, 3), dtype=np.float32)
    m = image_size-1
    for y in range(m):
        for x in range(m):
            if constant == 'hue':
                s = x / m
                i = y / m
                rgb = hsi_to_rgb_value(val, s, i)
            elif constant == 'saturation':
                h = x / m
                i = y / m
                rgb = hsi_to_rgb_value(h, val, i)
            elif constant == 'intensity':
                h = x / m
                s = y / m
                rgb = hsi_to_rgb_value(h, s, val)
            if np.any(rgb < 0) or np.any(rgb > 1):
                rgb = [1, 0, 0]  # Error color (red)
            image[y, x] = rgb
    return (image * 255).astype(np.uint8)

# Create images
hue_constant_image = create_hsi_image(image_size=400,constant='hue',val=0.9)
saturation_constant_image = create_hsi_image(image_size=400,constant='saturation',val=0.7)
intensity_constant_image = create_hsi_image(image_size=400,constant='intensity',val=0.4)

# Save images
cv2.imwrite('output_images/Q4/Q4_a_hue_constant.png', hue_constant_image)
cv2.imwrite('output_images/Q4/Q4_a_saturation_constant.png', saturation_constant_image)
cv2.imwrite('output_images/Q4/Q4_a_intensity_constant.png', intensity_constant_image)

# Display images
cv2.imshow('Hue Constant image', hue_constant_image)
cv2.imshow('Saturation Constant image', saturation_constant_image)
cv2.imshow('Intensity Constant image', intensity_constant_image)
cv2.waitKey(5000)
cv2.destroyAllWindows()


In [174]:
######********** Q4 c******************

In [175]:
#### RGB  image to HSI
def rgb_to_hsi(image:np.array):
    normalised_img = image /255.0
    I = np.sum(normalised_img,axis=2)/3.0
    r = normalised_img[:,:,0]
    g = normalised_img[:,:,1]
    b = normalised_img[:,:,2]
    S = 1 - (3*np.minimum(r,np.minimum(g,b))/(r+g+b +1e-9))
    hue_angle = np.arccos(0.5 * (2*r - g - b) / (np.sqrt( np.square(r-g) + (r-b)*(g-b) +1e-9 )))
    H = 0.5*np.where(b<=g,hue_angle,2*np.pi - hue_angle)/np.pi
    hsi = np.stack([H,S,I],axis=-1)
    return hsi

## #******** HSI image to hsi
def hsi_to_rgb_image(image:np.array):
    rgb_image = np.zeros(shape=image.shape)
    for x in range(image.shape[0]):
        for y in range(image.shape[1]):
            rgb = hsi_to_rgb_value(image[x,y,0],image[x,y,1],image[x,y,2])
            if np.any(rgb < 0) or np.any(rgb > 1):
                rgb = [1, 0, 0]  # Error color (red)
            image[x, y] = rgb
    return (image * 255.0)

In [176]:
def reduce_contrast(image:np.array):
    image[:,:,2] = 1e-4 + ((image[:,:,2]-np.min(image[:,:,2]))/(np.max(image[:,:,2] -np.min(image[:,:,2]))))*(1-1e-4)
    return image


In [177]:
pg = rgb_to_hsi(image4)
hg = reduce_contrast(pg)
print(np.min(hg))
rg = hsi_to_rgb_image(hg)
np.min(rg)

0.0


0.0

In [178]:
cv2.imshow('Intensity Constant image', rg)
cv2.waitKey(5000)
cv2.destroyAllWindows()

In [197]:
#***********
#************Q4 d************
def gamma_correction(image:np.array,gamma:float):
    # # image = image/255.0
    # print(np.max(image[:,:,0]))
    # gamma_corrected_image = image**gamma
    # print(np.min(image[:,:,0]))
    # gamma_corrected_image = 255*gamma_corrected_image
    # print(np.min(gamma_corrected_image))
    # print(np.max(gamma_corrected_image))

    # scaled_image = 255*((gamma_corrected_image - np.min(gamma_corrected_image))/(np.max(gamma_corrected_image) - np.min(gamma_corrected_image)))
    # return scaled_image
    return np.clip((image / 255.0) ** gamma * 255, 0, 255)




In [200]:
gg = gamma_correction(image4,(1/2.2))

In [201]:
cv2.imshow('Intensity Constant image', gg)
cv2.waitKey(5000)
cv2.destroyAllWindows()